# Real World Example: Plant Growth Experiment

## Scenario: Studying the Effects of Fertilizers
In this experiment, researchers are evaluating the impact of three different fertilizers on plant growth under controlled conditions. Weekly measurements of plant height are recorded for three groups of plants, each receiving a different fertilizer. The workflow includes:
- Uploading the collected data to openBIS.
- Analyzing the growth trends to determine the best-performing fertilizer.
- Archiving the data and updating metadata for future reference.

This example demonstrates how pyBIS can streamline data management and analysis for scientific experiments.

## Part 1: Data Creation

### Connecting to openBIS
First, establish a connection to your openBIS instance.

In [ ]:
from pybis import Openbis
o = Openbis('https://schulung.datastore.bam.de')
o.login('username', 'password')

### Creating a New Experiment
Define the space, project, and experiment for this study. If they don't already exist, create them.

In [ ]:
SPACE='PLANT_STUDY'
PROJECT='FERTILIZER_IMPACT'
EXPERIMENT='GROWTH_ANALYSIS'

try:
    space = o.get_space(SPACE)
except:
    space = o.new_space(SPACE, 'Plant Growth Studies')

try:
    project = space.get_project(PROJECT)
except:
    project = space.new_project(PROJECT, 'Impact of Fertilizers on Growth')

try:
    experiment = project.get_collection(EXPERIMENT)
except:
    experiment = project.new_collection(EXPERIMENT, 'Weekly Growth Measurements')

### Uploading Weekly Growth Data
Each week's measurements are saved in CSV format and uploaded as a dataset. For this example, we use the `plant_growth_data.csv` file as our source data.

In [ ]:
dataset = experiment.new_dataset(type='RAW_DATA', files=['plant_growth_data.csv'])

## Part 2: Data Analysis

### Fetching Data for Analysis
Retrieve datasets associated with the experiment for processing.

In [ ]:
datasets = experiment.get_datasets()
raw_data = [ds for ds in datasets if ds.type == 'RAW_DATA']
print(f'{len(raw_data)} raw datasets found.')

### Analyzing Growth Trends
Using the weekly measurements, calculate average growth per week for each fertilizer group and visualize the results.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('plant_growth_data.csv')
fertilizers = data['Fertilizer'].unique()

# Calculate average growth
growth_summary = data.groupby('Fertilizer')['Height'].mean()
growth_summary.plot(kind='bar', title='Average Growth by Fertilizer')
plt.ylabel('Average Height (cm)')
plt.savefig('growth_summary.png')

### Uploading Analysis Results
Save the summary and visualization back to openBIS.

In [ ]:
analysis_dataset = experiment.new_dataset(type='ANALYZED_DATA', files=['growth_summary.png'])

## Part 3: Maintenance

### Archiving Raw Data
To ensure long-term availability, archive the raw datasets locally.

In [ ]:
import os

archive_dir = './archive'
os.makedirs(archive_dir, exist_ok=True)

for ds in raw_data:
    ds.download(destination=archive_dir)

### Updating Metadata with Key Insights
After analysis, update the experiment metadata to include a summary of findings.

In [ ]:
experiment.props['Summary'] = 'Fertilizer B showed the highest average growth across all weeks.'
experiment.save()

## Conclusion
This example demonstrates how pyBIS can streamline the management of experimental data through:
- Efficient data upload and storage.
- Integrated analysis and visualization workflows.
- Archival and metadata updates for future reference.

By automating these steps, researchers can focus more on insights and less on data handling.

### Logout

In [ ]:
o.logout()